# The Problem

We are a librarian tasked with identifying waste in spending on expensive databases. We want to ensure that we only remove databases that have low quantity of uses with high quantity of ILL uses. We have data in two separatae csv files (generated here as a dictionary). Main is the main database that indicates local users accessing specific journals. ILL is the ILL csv file that indicates the frequency of that journal being ILLed. If users need to ILL a journal with a high frequency and are not able to leverage a local purchase of it to fill their needs, then we can presume that the payment for that journal is unnecessary. Our goal is to find the journals that have the highest percentage of non-use.

In [3]:
import pandas as pd

In [4]:
data_main = {"journal":["Anthropology Weekly", "History Monthly", "Science Quarterly", "Zoology Today"], "quantity": [18, 20, 200, 700]}
data_ill = {"journal":["Anthropology Weekly", "History Monthly", "Zoology Today"], "quantity": [2, 500, 1000]}

In [5]:
df_main = pd.DataFrame(data_main)
df_main

,journal,quantity
0,Anthropology Weekly,18
1,History Monthly,20
2,Science Quarterly,200
3,Zoology Today,700


In [6]:
df_ill = pd.DataFrame(data_ill)
df_ill

,journal,quantity
0,Anthropology Weekly,2
1,History Monthly,500
2,Zoology Today,1000


# The Solution outside of Pandas

## Identify the Areas of Overlap

In [7]:
main_journals = df_main.journal.tolist()
ill_journals = df_ill.journal.tolist()

In [8]:
overlap = set(main_journals).intersection(ill_journals)
print(overlap)

{'Zoology Today', 'Anthropology Weekly', 'History Monthly'}


## Isolate the Rows that are Important for Each DataFrame

In [13]:
data = {"journal": [], "quantity": [], "ill_quantity": []}
final_main = pd.DataFrame(data)
for o in overlap:
    print(o)
    for index, row in df_main.iterrows():      
        if row["journal"] == o:
            row['ill_quantity'] = None
            final_main = final_main.append(row, ignore_index=True)
    for index, row in df_ill.iterrows():
        if row["journal"] == o:
            for index2, row2 in final_main.iterrows():
                if row2['journal'] == o:
                    final_main.loc[index2, 'ill_quantity'] = row['quantity']

Zoology Today
Anthropology Weekly
History Monthly


In [14]:
final_main

,journal,quantity,ill_quantity
0,Zoology Today,700.0,1000
1,Anthropology Weekly,18.0,2
2,History Monthly,20.0,500


## Calculate the Ratio

In [15]:
final_main["result"] = final_main["quantity"]/final_main["ill_quantity"]

In [16]:
final_main

,journal,quantity,ill_quantity,result
0,Zoology Today,700.0,1000,0.7
1,Anthropology Weekly,18.0,2,9.0
2,History Monthly,20.0,500,0.04


## Organize the Data so that the Lowest Percentage Items are Highest

In [17]:
final_main.sort_values("result")

,journal,quantity,ill_quantity,result
2,History Monthly,20.0,500,0.04
0,Zoology Today,700.0,1000,0.7
1,Anthropology Weekly,18.0,2,9.0


# Solution with Pandas

In [18]:
df_main

,journal,quantity
0,Anthropology Weekly,18
1,History Monthly,20
2,Science Quarterly,200
3,Zoology Today,700


In [19]:
df_ill

,journal,quantity
0,Anthropology Weekly,2
1,History Monthly,500
2,Zoology Today,1000


## Merge Pandas DataFrames

In [29]:
final_df = pd.merge(df_main, df_ill, how='inner', on=['journal'])
final_df

,journal,quantity_x,quantity_y
0,Anthropology Weekly,18,2
1,History Monthly,20,500
2,Zoology Today,700,1000


## Rename Columns and Clean

In [30]:
final_df.columns = ["journal", "quantity", "ill_quantity"]

In [34]:
final_df["result"] = final_df["quantity"]/final_df["ill_quantity"]

In [35]:
final_df.sort_values("result")

,journal,quantity,ill_quantity,result
1,History Monthly,20,500,0.04
2,Zoology Today,700,1000,0.70
0,Anthropology Weekly,18,2,9.00
